# NCAA MARCH MADNESS - Machine Learning

## 1. Data Loading

Two groups to choose from: detailed results or compact results. Compact stretches back to 1985, detailed only to 2003. Detailed provides us with more features and less to parse through. Each record from datasets 'MRegularSeasonDetailedResults' and 'MNCAATourneyDetailedResults' represents a game from the year's regular season or the tournament with columns for winning and losing team's final scores, attempted FG, attempted 3-point shots, rebounds, fouls, etc.

We want one comprehensive dataset that we can then split for training and testing for our final model that includes features that we can use to predict success.

In [1]:
import pandas as pd
import seaborn as sns
import os

season = pd.read_csv(os.path.join("2020DataFiles", "MRegularSeasonDetailedResults.csv"))
tourney = pd.read_csv(os.path.join("2020DataFiles", "MNCAATourneyDetailedResults.csv"))
w_teams = pd.read_csv(os.path.join("2020DataFiles", "MTeams.csv"))
l_teams = pd.read_csv(os.path.join("2020DataFiles", "MTeams.csv"))
w_seeds = pd.read_csv(os.path.join("2020DataFiles", "MNCAATourneySeeds.csv"))
l_seeds = pd.read_csv(os.path.join("2020DataFiles", "MNCAATourneySeeds.csv"))
conf = pd.read_csv(os.path.join("2020DataFiles", "MConferenceTourneyGames.csv"))

In [1]:
season;

NameError: name 'season' is not defined

## Data Cleaning: 
Renaming columns in our preliminary dataframes. Make separate dataframes for winners and losers to make merging easier down the road

In [3]:
w_teams = w_teams.rename(columns={
    "TeamID":"WTeamID",
    "TeamName":"WTeamName",
    "FirstD1Season":"WFirstD1Season",
    'LastD1Season':"WLastD1Season"
})

l_teams = l_teams.rename(columns={
    "TeamID":"LTeamID",
    "TeamName":"LTeamName",
    "FirstD1Season":"LFirstD1Season",
    'LastD1Season':"LLastD1Season"
});

w_seeds = w_seeds.rename(columns={
    "TeamID":"WTeamID",
    "Seed":"WSeed"
})

l_seeds = l_seeds.rename(columns={
    "TeamID":"LTeamID",
    "Seed":"LSeed"
})

In [4]:
w_seeds['WSeed'] = w_seeds['WSeed'].str.strip('WXYZab').astype(int)

In [5]:
w_seeds = w_seeds.loc[w_seeds['Season'] == 2019]

In [6]:
w_seeds = w_seeds.merge(w_teams, on='WTeamID').drop(columns=['Season','WTeamID','WFirstD1Season','WLastD1Season'])

In [7]:
w_seeds = w_seeds.rename(columns={'WTeamName':'Name'})

In [8]:
w_seeds.head()

,WSeed,Name
0,1,Duke
1,2,Michigan St
2,3,LSU
3,4,Virginia Tech
4,5,Mississippi St


# CONFERENCE DATA:
Conference tourney champions

In [9]:
conf = conf.merge(w_teams, on="WTeamID").merge(l_teams, on="LTeamID")

conf = conf.drop(columns=['WFirstD1Season','WLastD1Season','LFirstD1Season','LLastD1Season','WTeamID','LTeamID'])

conf = conf.sort_values(by=["Season","ConfAbbrev"], axis=0).reset_index(drop=True)

conf_2019 = conf.loc[conf['Season'] == 2019]

In [10]:
conf_winners = pd.DataFrame(conf_2019.groupby(['ConfAbbrev'])['WTeamName'].value_counts())
conf_winners = conf_winners.rename(columns={'WTeamName':"Appearances"})
conf_winners = conf_winners.reset_index()

In [11]:
conf_winners.head()

,ConfAbbrev,WTeamName,Appearances
0,a_sun,Liberty,3
1,a_sun,Lipscomb,2
2,a_sun,NJIT,1
3,a_sun,North Florida,1
4,a_ten,St Louis,4


In [11]:
conf_win_count = pd.DataFrame(conf_winners.groupby(['ConfAbbrev'])['Appearances'].max())
conf_win_count.reset_index();
conf_win_count.head()

,Appearances
ConfAbbrev,
a_sun,3
a_ten,4
aac,3
acc,3
aec,3


In [12]:
conf_champs = conf_win_count.merge(conf_winners, on=['ConfAbbrev','Appearances'])

In [13]:
conf_champs.head()

,ConfAbbrev,Appearances,WTeamName
0,a_sun,3,Liberty
1,a_ten,4,St Louis
2,aac,3,Cincinnati
3,acc,3,Duke
4,aec,3,Vermont


# Season Data:
Merging Season df with Teams df to get a Team Name column and to create a "Years of Experience" column for how long a team has been coming to the tournament. Passing only the columns we want then sorting by Season and DayNum to see the season in sequential order

In [14]:
season = season.merge(w_teams, on="WTeamID")
season = season.merge(l_teams, on="LTeamID")

season["WYearsExp"] = season["WLastD1Season"] - season["WFirstD1Season"]

season["LYearsExp"] = season["LLastD1Season"] - season["LFirstD1Season"]

In [15]:
season = season[['Season','DayNum','WTeamName','LTeamName','WScore','LScore','WLoc',
                 'NumOT', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR', 
                 'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 
                 'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF',
                 'WYearsExp', 'LYearsExp']]

season = season.sort_values(by=["Season","DayNum"], axis=0).reset_index(drop=True)

For the sake of simplicity we split to only look at data for 2018-2019 season

In [16]:
season_2019 = season.loc[season['Season'] == 2019]

In [17]:
winners = season_2019.drop(columns=['LTeamName', 'LScore', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3',
                 'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF',
                'LYearsExp','LYearsExp', 'Season', 'DayNum', 'NumOT', 'WLoc', 'WFGA',
                'WFGA3', 'WFTM'])

winners = winners.rename(columns={'WTeamName': 'Name', 'WScore': 'Points', 'WFGM': 'FGM', 'WFGM3': 'FGM3', 'WStl': 'Steals',
                                  'WBlk': 'Blocks', 'WPF': 'Fouls', 'WOR': 'Off Rebounds', 'WDR': 'Def Rebounds',
                                  'WAst': 'Assists', 'WTO': 'Turnovers','WYearsExp': 'YearsExp', 'WFTA': 'FTA'})

losers = season_2019.drop(columns=['WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR',
                 'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'Season', 'DayNum', 'NumOT','WTeamName',
                'WScore','WLoc','WYearsExp', 'LFGA', 'LFGA3', 'LFTM'])

losers = losers.rename(columns={'LTeamName': 'Name', 'LScore': 'Points', 'LFGM': 'FGM', 'LFGM3': 'FGM3', 'LStl': 'Steals',
                                'LBlk': 'Blocks','LPF': 'Fouls','LOR': 'Off Rebounds', 'LDR': 'Def Rebounds',
                                'LAst': 'Assists', 'LTO': 'Turnovers', 'LYearsExp': 'YearsExp', 'LFTA': 'FTA'})

full_season = winners.append(losers);
full_season;

wins = winners.groupby('Name').count()
wins = wins.drop(columns=['FGM', 'FGM3', 'FTA', 'Steals', 'Blocks', 'Fouls', 'YearsExp', 'Off Rebounds',
                          'Def Rebounds', 'Assists', 'Turnovers']).rename(columns={'Points': 'SeasonWins'})
wins;

season_grouped = full_season.groupby(['Name'])
average_stats = season_grouped.mean()
seas = pd.merge(wins, average_stats, on='Name').reset_index()

In [18]:
seas = seas.assign(Conf_Champs=seas.Name.isin(conf_champs.WTeamName).astype(int))

In [19]:
seas;

In [20]:
full_seas = seas.merge(w_seeds, how='outer', on='Name').fillna(0)

tourney_seas = seas.merge(w_seeds, on='Name').fillna(0)

In [21]:
tourney_seas;

# TOURNEY DATA:
Merging Tourney with Teams for same reasons as above. When we merge Tourney with the Seed table to retrieve seeding information, we merge on two keys.

In [22]:
tourney = tourney.merge(w_teams, on="WTeamID")
tourney = tourney.merge(l_teams, on="LTeamID")

tourney["WYearsExp"] = tourney["WLastD1Season"] - tourney["WFirstD1Season"]

tourney["LYearsExp"] = tourney["LLastD1Season"] - tourney["LFirstD1Season"]

In [23]:
tourney = tourney[['Season', 'DayNum', 'WTeamName', 'LTeamName',  'WScore', 'LScore', 'WLoc',
       'NumOT', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR',
       'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3',
       'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF',
        'WYearsExp', 'LYearsExp']]

tourney = tourney.sort_values(by=["Season","DayNum"], axis=0).reset_index(drop=True)

In [24]:
tourney_2019 = tourney.loc[tourney['Season'] == 2019]

In [25]:
tourney_2019 = tourney_2019[tourney_2019.DayNum >= 136]

In [26]:
tourney_2019;

In [27]:
temp = tourney_2019[['Season', 'WTeamName']].rename(columns={'WTeamName': 'Name'})
tourney_wins = temp.groupby('Name').count()
tourney_wins = tourney_wins.reset_index().rename(columns={'Season': 'TourneyWins'})
tourney_wins;

### FINAL MERGE FOR DATASET WITH TOURNAMENT WINS

In [28]:
full_seas = full_seas.merge(tourney_wins, how='outer', on = 'Name').fillna(0)

In [29]:
tourney_seas = tourney_seas.merge(tourney_wins, how= 'outer', on = 'Name').fillna(0)

In [30]:
full_seas.to_csv(os.path.join("final_dataset353.csv"))

In [31]:
tourney_seas.to_csv(os.path.join("final_dataset68.csv"))